<a href="https://colab.research.google.com/github/mantoan-thi/Aprendizagem-por-Refor-o-com-Augmented-Random-Search-ARS-/blob/main/Aprendizagem_por_Refor%C3%A7o_com_Augmented_Random_Search_(ARS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np


class Hp():
  def __init__(self):
    self.nb_steps = 1000         # número de loops que vamos utilizar para o treinamento.
    self.episode_lenght = 1000   # máxima duração de um epsódio.
    self.learning_rate = 0.002   # Taxa de aprendizagem.
    self.nb_directions = 16      # número de direções -  quantidade de matrizes...
    self.nb_best_directions = 16 # seleciona as melhores matrizes.
    assert self. nb_best_directions <= self.nb_directions # nb_best_directions tem que ser menor ou igual ao self.nb_directions.
    self.noise = 0.03            # Usado para não termos variância nos dados.
    self.seed = 1                # usamos para termos os mesmos resultados.
    self.env_name = ''           # nome do ambiente.

class Normalizer():
  def__init__(self,nb_inputs):
    self.n = np.zeros(np_inputs)          # Quantidade de entradas.
    self.mean = np.zeros(np_inputs)       # Média
    self.mean_diff = np.zeros(np_inputs)  # 
    self.var = np.zeros(np_inputs)        # Variância